In [ ]:
import os
import ast
import re
import json
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import time
import gc
import torch
import torch.nn.functional as F

from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/x_train_Meacfjr.csv", delimiter=",", quotechar='"')
y_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/y_train_SwJNMSu.csv", delimiter=",", quotechar='"')

# Convert 'job_ids' and 'actions' from strings to actual lists
X_train["job_ids"] = X_train["job_ids"].apply(ast.literal_eval)
X_train["actions"] = X_train["actions"].apply(ast.literal_eval)

df_jobs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_features.csv")

df_job_location = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_features_location.csv")

In [ ]:
for _, row in df_job_location.iterrows():
    df_jobs.loc[df_jobs['job_id'] == row['job_id'], 'location'] = row['location']

print(df_jobs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21917 entries, 0 to 21916
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_id           21917 non-null  int64 
 1   title_section    21913 non-null  object
 2   job_description  21917 non-null  object
 3   seniority        21897 non-null  object
 4   company          21917 non-null  object
 5   industry         21905 non-null  object
 6   location         21916 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.2+ MB
None


In [ ]:
display(HTML(df_jobs[["job_id",	"title_section",	"seniority",	"company",	"industry",	"location"]].sample(200).to_html()))

,job_id,title_section,seniority,company,industry,location
20691,25955,Architecte messagerie M365,Senior,LeHibou,Banking,"Bordeaux, France"
15524,19722,UX DESIGNER SENIOR,Senior,Computer Futures,IT,"London, England"
1490,1492,Chef de projet technique média,Mid,"""MediaCorp""",Media,"New York, USA"
3653,3681,Développeur Full Stack Java/ Angular,Senior,Développeur Full Stack Java/Ang,Full Stack,"Paris, France"
21865,27295,Développeur BO BI4,Senior,SAP,Industry: Finance,"Paris, France"
6212,6438,Architecte GCP,Senior,Google Cloud Platform,IT,"Paris, France"
21307,26633,Product Manager PIM/DAM/MDM,Senior,Carrefour Group,Industry: Retail,"Paris, France"
21479,26818,Consultant project manager Senior,Senior,Recueillir et comprendre les besoins,IT,"New York, USA"
19937,25011,un architecte cloud Azure F/H,Senior,Infogene,Cloud,"Paris, France"
15369,19533,Lead Tech - Javascript (h/f),Senior,LeHibou,IT,"Bordeaux, France"


## GPU Requirements

In [ ]:
!nvcc --version # find the CUDA driver build above

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Mar 13 06:58:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## 1.3 Embedding of the Job Ads

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

response = client.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-3-large",
    dimensions=1024
)

print(response.data[0].embedding)

[0.005138647276908159, -0.01411565113812685, -0.0042228177189826965, 0.06196383386850357, 0.02284259907901287, -0.013790578581392765, -0.04558517783880234, 0.06381424516439438, -0.018079036846756935, 0.020267024636268616, 0.024693012237548828, 0.033707525581121445, -0.03988390415906906, -0.012559053488075733, -0.01426568441092968, 2.0878684154013172e-05, -0.028906453400850296, -0.005501227919012308, -0.03810850903391838, -0.033707525581121445, 0.042459480464458466, 0.027831213548779488, -0.09812190383672714, 0.0516115203499794, -0.00618575606495142, 0.021754857152700424, -0.04918598011136055, 0.016141103580594063, 0.015515963546931744, -0.016353650018572807, 0.04863585904240608, 0.029806654900312424, 0.045310113579034805, 0.005998214241117239, -0.0030350524466484785, -0.020967179909348488, 0.025530699640512466, 0.05266175791621208, 0.0035320385359227657, 0.027131056413054466, 0.005651261657476425, 0.009020763449370861, -0.0645144060254097, 0.0030334896873682737, 0.01237151212990284, 0.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define features and weights
features = ['title_section', 'location', 'seniority', 'company', 'industry']  # Example with multiple features

In [ ]:
def get_openai_embeddings(texts, model="text-embedding-3-small", dimensions=1024):
    """
    Fetch embeddings from OpenAI API in batches.

    :param texts: List of input texts.
    :param model: OpenAI embedding model.
    :param dimensions: Embedding size.
    :return: List of embedding tensors.
    """
    response = client.embeddings.create(input=texts, model=model, dimensions=dimensions)
    return [torch.tensor(item.embedding, device="cuda") for item in response.data]

In [ ]:
# from sentence_transformers import SentenceTransformer

# Load pre-trained sentence transformer model and move it to the GPU if available
# model = SentenceTransformer('all-MiniLM-L6-v2').to(device)  # intfloat/multilingual-e5-large




def compute_job_embeddings(df, features, batch_size=10, embedding_dim=1024, display_every=15, save_every=100):
    """
    Compute embeddings feature by feature, batching API requests for efficiency.

    :param df: Pandas DataFrame containing job data.
    :param features: List of feature names.
    :param batch_size: Number of rows per batch.
    :param embedding_dim: Expected size of embeddings.
    :return: Tuple (job_embeddings, mask_vectors).
    """
    job_embeddings = {feature: [] for feature in features}
    job_embeddings_norm = {feature: [] for feature in features}
    mask_vectors = {feature: [] for feature in features}

    for feature in features:
        print(f"Processing feature: {feature}")

        # Convert feature column to a list
        feature_texts = df[feature].astype(str).fillna("").tolist()

        # Identify valid (non-missing) entries
        valid_mask = [(text != "" and len(text) > 1) for text in feature_texts]

        # Create placeholder tensors for embeddings
        feature_embeddings = [torch.zeros(embedding_dim, device="cuda") for _ in range(len(df))]

        # Process in batches
        for i in tqdm(range(0, len(df), batch_size), desc=f"Embedding {feature}"):
            start_time = time.time()
            batch_texts = [feature_texts[j] for j in range(i, min(i + batch_size, len(df))) if valid_mask[j]]
            batch_indices = [j for j in range(i, min(i + batch_size, len(df))) if valid_mask[j]]

            if batch_texts:
                batch_embeddings = get_openai_embeddings(batch_texts)
                for idx, emb in zip(batch_indices, batch_embeddings):
                    feature_embeddings[idx] = emb

        # Store embeddings and mask
        job_embeddings[feature] = torch.stack(feature_embeddings)
        mask_vectors[feature] = torch.tensor(valid_mask, dtype=torch.float32, device=device).view(-1, 1)  # 1 for valid, 0 for missing

        # Save feature
        print(f"Saving featuere {feature}...")
        path = f"/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small_checkpoint.pth"
        torch.save(job_embeddings, path)

        # Zero mean
        valid_embeddings = job_embeddings[feature] * mask_vectors[feature]  # Apply mask
        mean = valid_embeddings.sum(dim=0) / mask_vectors[feature].sum()  # Compute mean ignoring missing rows
        job_embeddings_norm[feature] = (job_embeddings[feature] - mean) * mask_vectors[feature]  # Normalize only valid rows

        # Unit norm
        norm = torch.norm(job_embeddings_norm[feature], dim=1, keepdim=True)  # Compute L2 norm
        norm = torch.where(norm == 0, torch.ones_like(norm), norm)  # Avoid division by zero
        job_embeddings_norm[feature] = job_embeddings_norm[feature] / norm  # Normalize

    return job_embeddings, mask_vectors



    return job_embeddings, job_embeddings_norm

In [ ]:
# Load or Compute Embeddings
embedding_path = "/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small.pth"
embedding_path_norm = "/content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small_norm.pth"
if os.path.exists(embedding_path):
    print("Loading embeddings from {}".format(embedding_path))
    job_embeddings = torch.load(embedding_path, map_location=device)
    job_embeddings_norm = torch.load(embedding_path_norm, map_location=device)
else:
    print("Computing embeddings...")
    job_embeddings, job_embeddings_norm = compute_job_embeddings(df_jobs, features)
    torch.save(job_embeddings, embedding_path)
    torch.save(job_embeddings_norm, embedding_path_norm)

# Add embeddings to DataFrame
for feature in features:
    df_jobs[f'embedding_{feature}'] = list(job_embeddings[feature].cpu())

# Create a mapping from job_id to its embeddings (per feature)
job_embedding_dict = {row['job_id']: {feature: row[f'embedding_{feature}'] for feature in features}
                      for _, row in df_jobs.iterrows()}

Loading embeddings from /content/drive/MyDrive/Colab Notebooks/Challenge/data/job_embeddings_small.pth


<ipython-input-28-60fee70f21a4>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  job_embeddings = torch.load(embedding_path, map_location=device)
<ipython-input-28-60fee70f2

In [ ]:
def preprocess_embeddings(df_jobs, job_embedding_dict):
    """ Convert job embeddings into a single tensor for fast access. """
    job_id_to_idx = {job_id: idx for idx, job_id in enumerate(df_jobs['job_id'].values)}

    job_embeddings_tensor = torch.stack([
        torch.stack([job_embedding_dict[job_id][feature] * feature_weights[feature] for feature in features]).sum(dim=0)
        for job_id in df_jobs['job_id']
    ]).to(torch.float32).to('cuda')  # Shape: (num_jobs, embedding_dim)

    job_ids_tensor = torch.tensor(df_jobs['job_id'].values, dtype=torch.long, device='cuda')  # (num_jobs,)

    return job_embeddings_tensor, job_ids_tensor, job_id_to_idx

def preprocess_sessions(x, job_id_to_idx, action_weights):
    """ Convert session interactions into tensor format for batch processing. """
    max_jobs_per_session = max(len(job_ids) for job_ids in x['job_ids'])

    session_ids = torch.tensor(x['session_id'].values, dtype=torch.long, device='cuda')

    job_indices = torch.full((len(x), max_jobs_per_session), -1, dtype=torch.long, device='cuda')
    action_weights_tensor = torch.zeros((len(x), max_jobs_per_session), dtype=torch.float32, device='cuda')

    for i, (job_ids, actions) in enumerate(zip(x['job_ids'], x['actions'])):
        valid_indices = [job_id_to_idx[jid] for jid in job_ids if jid in job_id_to_idx]
        job_indices[i, :len(valid_indices)] = torch.tensor(valid_indices, dtype=torch.long, device='cuda')
        action_weights_tensor[i, :len(valid_indices)] = torch.tensor(
            [action_weights[a] for a in actions[:len(valid_indices)]], dtype=torch.float32, device='cuda'
        )

    return session_ids, job_indices, action_weights_tensor


In [ ]:
def loss_mrr(y_true, y_pred, verbose=True):
    """
    Computes Mean Reciprocal Rank (MRR).

    Args:
        y_true: dataframe of true job_id values per session.
        y_pred: Predicted action and 10 most relevant job_ids per session.

    Returns:
        MRR final score (float).
    """
    y_true_jobs = y_true["job_id"]
    reciprocal_ranks = []

    for true_job, (action, predicted_jobs) in zip(y_true_jobs, y_pred):
        try:
            rank = predicted_jobs.index(true_job) + 1  # Get 1-based rank
            reciprocal_ranks.append(1 / rank)
        except ValueError:
            reciprocal_ranks.append(0)  # y_true not found in top-10

    reciprocal_ranks_tensor = torch.tensor(reciprocal_ranks, dtype=torch.float32)

    # Count occurrences using torch (no Counter)
    unique_values, counts = torch.unique(reciprocal_ranks_tensor, return_counts=True)

    if verbose:
        # Display value counts
        print("\nReciprocal Rank Value Counts:")
        for val, count in zip(unique_values.tolist(), counts.tolist()):
            print(f"  {val:.4f}: {count} occurrences")
        print("\nMRR:", reciprocal_ranks_tensor.mean().item())

    return reciprocal_ranks_tensor.mean().item()

## Recommend list of the 10 most relevant jobs

In [ ]:
feature_weights = {
    'title_section': 1.0,
    'location': 0.8,
    'seniority': 0.3,
    'company': 0.3,
    'industry': 0.8
    }

action_weights = {'apply': 5, 'view': 1}

In [ ]:
# Function to generate recommendations for sessions dataframe using GPU
def recommend_jobs_for_sessions(x, df_jobs, job_embedding_dict, feature_weights, action_weights, top_n=10, batch_size=64):
    """ Fully parallelized job recommendation system using PyTorch tensor operations. """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Precompute job embeddings and indices for fast access
    job_embeddings_tensor, job_ids_tensor, job_id_to_idx = preprocess_embeddings(df_jobs, job_embedding_dict)

    # Preprocess session interactions
    session_ids, job_indices, action_weights_tensor = preprocess_sessions(x, job_id_to_idx, action_weights)

    all_recommendations = []

    for batch_start in range(0, len(x), batch_size):
        if (batch_start // batch_size + 1) % 20 == 0:
            print("Processing batch", batch_start // batch_size + 1, "of", len(x) // batch_size + 1)
        start_time = time.time()
        batch_end = min(batch_start + batch_size, len(x))
        batch_job_indices = job_indices[batch_start:batch_end]
        batch_action_weights = action_weights_tensor[batch_start:batch_end]

        # Mask out invalid job indices (-1)
        valid_mask = batch_job_indices != -1
        batch_job_embeddings = job_embeddings_tensor[batch_job_indices.clamp(min=0)] * valid_mask.unsqueeze(-1)

        # Compute user profiles in parallel
        batch_weighted_sums = (batch_job_embeddings * batch_action_weights.unsqueeze(-1)).sum(dim=1)
        batch_action_sums = batch_action_weights.sum(dim=1, keepdim=True).clamp(min=1e-6)
        batch_user_profiles = batch_weighted_sums / batch_action_sums  # (batch_size, embedding_dim)

        # Compute cosine similarity in parallel
        similarities = F.cosine_similarity(batch_user_profiles.unsqueeze(1), job_embeddings_tensor.unsqueeze(0), dim=-1)

        # Exclude jobs already interacted with
        batch_seen_jobs = (batch_job_indices.unsqueeze(-1) == job_ids_tensor).any(dim=1)
        similarities[batch_seen_jobs] = -float('inf')  # Mask seen jobs

        # Get top-N recommendations
        top_recommendations = similarities.topk(top_n, dim=-1).indices  # (batch_size, top_n)

        # Convert to list format
        for i in range(batch_start, batch_end):
            session_id = session_ids[i].item()
            recommended_job_ids = job_ids_tensor[top_recommendations[i - batch_start]].tolist()
            all_recommendations.append((session_id, recommended_job_ids))

        del batch_user_profiles, similarities, batch_seen_jobs
        torch.cuda.empty_cache()
        gc.collect()

    return all_recommendations

In [ ]:
x, y = X_train, y_train
y_pred = recommend_jobs_for_sessions(x, df_jobs, job_embedding_dict, feature_weights, action_weights, batch_size=32)

Processing batch 20 of 497
Processing batch 40 of 497
Processing batch 60 of 497
Processing batch 80 of 497
Processing batch 100 of 497
Processing batch 120 of 497
Processing batch 140 of 497
Processing batch 160 of 497
Processing batch 180 of 497
Processing batch 200 of 497
Processing batch 220 of 497
Processing batch 240 of 497
Processing batch 260 of 497
Processing batch 280 of 497
Processing batch 300 of 497
Processing batch 320 of 497
Processing batch 340 of 497
Processing batch 360 of 497
Processing batch 380 of 497
Processing batch 400 of 497
Processing batch 420 of 497
Processing batch 440 of 497
Processing batch 460 of 497
Processing batch 480 of 497


In [ ]:
loss_mrr(y, y_pred)


Reciprocal Rank Value Counts:
  0.0000: 15511 occurrences
  0.1000: 22 occurrences
  0.1111: 27 occurrences
  0.1250: 34 occurrences
  0.1429: 36 occurrences
  0.1667: 41 occurrences
  0.2000: 50 occurrences
  0.2500: 33 occurrences
  0.3333: 43 occurrences
  0.5000: 41 occurrences
  1.0000: 44 occurrences

MRR: 0.007461877539753914


0.007461877539753914